## NLP Praktisches Notebook

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/Jona-Bach/llm-notebooks/blob/main/nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

### Sentiment Analysis


**Klassifier**

Das englische Modell Schaut nur nach Positiv oder Negativ

Das multilinguale Modell gibt 1 - 5 Sterne

In [ ]:
from transformers import pipeline

english_model = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
multi_lingual_model = "nlptown/bert-base-multilingual-uncased-sentiment"
classifier = pipeline("sentiment-analysis", model= english_model, framework="pt")
multi_classifier = pipeline("sentiment-analysis", model= multi_lingual_model, framework="pt")

text = "Trees are green"
text2 = "I hate this boat"

result = classifier(text)
result_multi = multi_classifier(text2)
print(text, result)
print(text2, result_multi)

**German**

In [ ]:
#!pip install germansentiment

In [ ]:
from germansentiment import SentimentModel

sent_model = SentimentModel()

text = ["Der Tag ist grün und die Sterne lila"] # Der Text muss in einer Liste übergeben werden, es können auch mehrere Sätze analysiert werden

result, probability = sent_model.predict_sentiment(text, output_probabilities=True)
print(result, probability)

---

### Thema 4: **Aktuelle Tools**


### Huggingface

#### Google Flan (Text2Text)

In [ ]:
from huggingface_hub import hf_hub_download

model_id = "google/flan-t5-base"
# model_id = "google/flan-t5-small" # Dümmer aber schneller
filenames = ["pytorch_model.bin","config.json","generation_config.json","special_tokens_map.json","spiece.model","tokenizer_config.json"]
for file in filenames:
    downloaded_model_path = hf_hub_download(
        repo_id=model_id,
        filename = file,
    )
    print(downloaded_model_path)

**Achtung das laden der Pipeline kann etwas dauern!**

Am besten Englisch verwenden. Text2Text Modelle sind gut für Aufgaben wie Zusammenfassungen, Übersetzungen oder Aufgabenlösung!

**e** drücken zum beenden!

In [ ]:
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

generator = pipeline("text2text-generation", model=model, device=-1, tokenizer=tokenizer)

while True:
    question = input("Give me a Task(e zum beenden): ")
    answer = generator(question)
    print(answer[0]["generated_text"])

    if question.lower() == "e":
        print("Beenden!")

#### Tiny Llama (Text Generation)

In [ ]:
from huggingface_hub import hf_hub_download

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
filenames = [
    "model.safetensors",
    "config.json",
    "eval_results.json",
    "tokenizer_config.json",
    "tokenizer.json",
    "special_tokens_map.json",
    "tokenizer.model",
    "generation_config.json"
]

for file in filenames:
    downloaded_model_path = hf_hub_download(
        repo_id=model_id,
        filename=file,
    )
    print(f"{file} -> {downloaded_model_path}")

In [ ]:
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=-1)

question = "Q: Where is the Buckingham Palace? \nA:"

response = generator(
    question,
    max_new_tokens=50,          # Begrenzung!
    do_sample=True,              # zufälligere Antworten
    temperature=0.7,             # Kreativität
    top_p=0.9                    # typische Sampling-Kombi
)

print(response[0]["generated_text"])

### Ollama ( + Langchain)

Wir verwenden hier das *llama3.2:3b* Modell von Meta

Dies kann man sich runterladen durch: **Ollama pull llama:3.2:3b** (Ollama muss vorher installiert werden)

**e** zum beenden!

In [ ]:
from langchain_ollama.llms import OllamaLLM
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

model_name = "llama3.2:3b" # 3 Millarden Parameter
llm = OllamaLLM(model=model_name)

template = "Beantworte diese Frage direkt und Präzise: \n{frage}"
prompt = PromptTemplate(input_variables=["frage"], template=template)

chain = prompt | llm


frage = "Wo steht der Buckingham Palace?"

antwort = chain.invoke({"frage": frage})
print(antwort)

# while True:

#     input_user = input("Stelle eine Frage: ")

#     if input_user.lower() == "e":
#         print("Beenden")
#         break
#     antwort = chain.invoke({"frage": input_user})
#     print(antwort)
